# Northern Ireland UBI Feasibility Study

*Research supported by the UBI Lab Northern Ireland.*

* Amount: £200, 300, 400 per month
* Half shares for children and pensioners
* UBI included in neither means tests nor income tax
* Personal tax allowance at £2k
* Other rates from there, prioritizing higher 

## Tax and benefit policy in Northern Ireland


In [1]:
import plotly.express as px
from ubicenter import format_fig

format_fig(
    px.bar(x=[1, 2, 3], y=[4, 6, 7]).update_layout(
        title="Plot title", xaxis_title="x-axis", yaxis_title="y-axis"
    )
)


## Costing the UBI

* Cost of UBI up to £400 per month
* Start with amount from personal allowance
* Show equivalence between each rate increase


In [2]:
from policyengine.countries import UK
from openfisca_uk.model_api import *
from openfisca_uk.api import *
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

uk = UK()


CHILD_PENSIONER_SHARE = 0.5


def change_tax_parameters(parameters):
    parameters.tax.income_tax.rates.uk.brackets[0].rate.update(
        period=periods.period("year:2022:10"), value=0.23
    )
    return parameters


class ubini(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


In [3]:
res = uk.population_reform(ubini)
res["net_cost"]


'-£13.7bn'

In [4]:
baseline, reformed = uk._get_microsimulations(ubini, refresh_baseline=True)

NI = "NORTHERN_IRELAND"


def filter_weights(sim):
    sim.set_input(
        "household_weight",
        sim.year,
        np.where(sim.calc("country") == NI, sim.calc("household_weight"), 0),
    )
    sim.set_input(
        "person_weight",
        sim.year,
        np.where(
            sim.calc("country", map_to="person") == NI,
            sim.calc("person_weight"),
            0,
        ),
    )


filter_weights(baseline)
filter_weights(reformed)


TypeError: _get_microsimulations() got an unexpected keyword argument 'refresh_baseline'

In [ ]:
res = uk.population_reform(ubini)
res["net_cost"]


'£1.09tr'

## Impacts of budget-neutral UBI plans

* Poverty
* Inequality
